In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
from tensorflow.keras import layers, models
from keras_tuner import HyperModel

In [5]:
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))

    # Tune the dropout rate
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    model.add(layers.Dropout(rate=hp_dropout))

    model.add(layers.Dense(10, activation='softmax'))

    # Compile the model
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [6]:
from keras_tuner import RandomSearch

In [7]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # You can increase this number for a more exhaustive search
    executions_per_trial=3,
    directory='my_dir',
    project_name='fashion_mnist'
)

In [8]:
tuner.search(x_train, y_train, epochs=10, validation_split=0.2)

Trial 5 Complete [00h 08m 08s]
val_accuracy: 0.8868333299954733

Best val_accuracy So Far: 0.8891666730244955
Total elapsed time: 00h 32m 44s


In [9]:
best_model = tuner.get_best_models(num_models=1)[0]
loss, accuracy = best_model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy}")

313/313 [==============================] - 1s 3ms/step - loss: 0.3438 - accuracy: 0.8815
Test accuracy: 0.8815000057220459


## THNAK YOU --- \\(^o^)/